In [9]:
import torch, numpy
from torch.func import hessian
from torch.func import jacrev
# I am going to use the optimzer from torch.
# pip install pytorch-minimize
from torchmin import minimize

def f(x):
    """ Gen Rosenbrock function.
    Args: x (torch.Tensor): Input tensor of shape (n).
    Returns: torch.Tensor: Output tensor of shape (1).
    """
    return torch.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2, dim=-1)

def dfSQ(x):
    """ Sum of squares of gradient for f
    Args: x: torch tensor of length n.
    Returns: torch scalar.
    """
    # Computes reverse mode gradient using PyTorch
    g=jacrev(f)(x)
    g=torch.square(g)
    return torch.sum(g)

In [94]:
# Define a random IC
n = 10
x = torch.randn(n)
# Minimze sum of squares
xMin=minimize(dfSQ, x, method="trust-exact")
print(xMin)

 message: A bad approximation caused failure to predict improvement.
 success: False
  status: 2
     fun: tensor(4.6619)
       x: tensor([-0.8037,  0.6571,  0.4377,  0.1920,  0.0167, -0.8937,  1.3094,  1.9108,
                   3.7047, 13.7354])
     nit: 241
    grad: tensor([ 2.8408e-02,  4.1148e-02, -2.1794e-02,  4.4679e-03,  2.5861e-04,
                  -1.4877e-04, -5.7561e-01, -1.6743e+01,  3.0460e+02, -4.0867e+01])
    nfev: 243
    hess: tensor([[ 7.3556e+05,  6.8108e+05, -1.6900e+05,  0.0000e+00,  0.0000e+00,
                    0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                  [ 6.8108e+05,  9.3908e+05, -4.7325e+05,  9.2049e+04,  0.0000e+00,
                    0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                  [-1.6900e+05, -4.7325e+05,  4.5176e+05, -2.0825e+05,  2.6892e+04,
                    0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                  [ 0.0000e+00,  9.2049e+04, -2.0

In [93]:
# Compute hessian 
g = jacrev(f)(xMin.x)
hess=hessian(f)(xMin.x)
GradRes = torch.linalg.norm(g)
print(GradRes)
# Compute eigenvalues
Lambdas=torch.linalg.eigvalsh(hess)
AppInvLambdas=torch.linalg.eigvalsh(xMin.hess_inv)
# sort and compare eigenvalues
torch.sort(Lambdas)
AppLambdas = torch.reciprocal(AppInvLambdas)
torch.sort(AppLambdas)
[AppLambdas, Lambdas]

tensor(2.0793)


AttributeError: hess_inv